# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121397e+02     1.713703e+00
 * time: 0.7272260189056396
     1     1.101098e+01     1.018616e+00
 * time: 3.029141902923584
     2    -1.202689e+01     9.442263e-01
 * time: 3.7162749767303467
     3    -3.427796e+01     7.169581e-01
 * time: 4.745611906051636
     4    -4.794666e+01     5.388025e-01
 * time: 5.777898073196411
     5    -5.695093e+01     1.998590e-01
 * time: 6.72732400894165
     6    -5.978442e+01     1.265642e-01
 * time: 7.401260852813721
     7    -6.086881e+01     5.456964e-02
 * time: 8.025332927703857
     8    -6.129001e+01     7.938306e-02
 * time: 8.691473007202148
     9    -6.162333e+01     3.502613e-02
 * time: 9.357320070266724
    10    -6.185245e+01     2.610485e-02
 * time: 10.009574890136719
    11    -6.202438e+01     2.252671e-02
 * time: 10.656172037124634
    12    -6.208887e+01     1.645504e-02
 * time: 11.282667875289917
    13    -6.214682e+01     1.313475e-02
 * time: 11.951510906219482
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557684
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666460462
